## Netflix DB

In [3]:
# Import libraries

import pandas as pd
from sqlalchemy import create_engine
import os
from config import Posgres_Pswrd

In [8]:
# Loading CSV file into Pandas

csv_file = os.path.join("netflix_titles.csv")
netflix_df = pd.read_csv(csv_file)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [5]:
# Exploring Netflix_titles file

netflix_df.shape

(6234, 12)

In [6]:
sorted(netflix_df)

['cast',
 'country',
 'date_added',
 'description',
 'director',
 'duration',
 'listed_in',
 'rating',
 'release_year',
 'show_id',
 'title',
 'type']

In [7]:
# Identifing missing fields

netflix_df['director'].isnull().sum()

1969

In [9]:
netflix_df['cast'].isnull().sum()

570

In [10]:
netflix_df['country'].isnull().sum()

476

In [11]:
netflix_df['date_added'].isnull().sum()

11

In [12]:
netflix_df['rating'].isnull().sum()

10

In [13]:
# Renaming columns

netflix_df.columns = ['show_id', 'show_type', 'show_title', 'show_director', 'show_cast', 'show_country', 'date_added', 'release_year', 'show_rating', 'show_duration', 'listed_in', 'show_description']
netflix_df.head()

,show_id,show_type,show_title,show_director,show_cast,show_country,date_added,release_year,show_rating,show_duration,listed_in,show_description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [14]:
# Assigning data types

# netflix_df.dtypes
netflix_df.astype({'date_added': 'datetime64[ns]'}).dtypes

show_id                      int64
show_type                   object
show_title                  object
show_director               object
show_cast                   object
show_country                object
date_added          datetime64[ns]
release_year                 int64
show_rating                 object
show_duration               object
listed_in                   object
show_description            object
dtype: object